# Phase 4.2 Experimental Lab: Multimodal Fusion Breakdown

**Goal:** CLIP-style alignment → swap modalities → observe confusion patterns

**Break it:** Misaligned training pairs → measure semantic drift

**Visualize:** Embedding space t-SNE plots, cross-modal similarity matrices

**Optimize:** Contrastive learning improvements, modality alignment techniques

---

## Professor's Notes

St. Mark, multimodal systems are powerful but fragile. When you combine text and images:
- **Alignment matters** - Misaligned pairs create semantic drift
- **Modalities interfere** - Corruption in one modality affects the other
- **Fusion is lossy** - Information gets lost in translation between modalities

Understanding these failure modes helps you build more reliable multimodal systems.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import make_blobs
import pandas as pd
from typing import List, Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🔬 Phase 4.2: Multimodal Fusion Breakdown")
print("=" * 50)

## 1. Build CLIP-Style Multimodal System

Let's create a simplified CLIP-like system that aligns text and image embeddings.

In [ ]:
class SimpleCLIPSystem:
    """Simplified CLIP-like multimodal alignment system"""
    
    def __init__(self, embedding_dim: int = 128, temperature: float = 0.07):
        self.embedding_dim = embedding_dim
        self.temperature = temperature
        self.text_embeddings = None
        self.image_embeddings = None
        self.text_descriptions = []
        self.image_labels = []
        
    def add_paired_data(self, text_embeddings: np.ndarray, image_embeddings: np.ndarray,
                       text_descriptions: List[str], image_labels: List[str]):
        """Add paired text-image data"""
        self.text_embeddings = text_embeddings.copy()
        self.image_embeddings = image_embeddings.copy()
        self.text_descriptions = text_descriptions
        self.image_labels = image_labels
        
        # Normalize embeddings
        self.text_embeddings = self.text_embeddings / np.linalg.norm(self.text_embeddings, axis=1, keepdims=True)
        self.image_embeddings = self.image_embeddings / np.linalg.norm(self.image_embeddings, axis=1, keepdims=True)
        
        print(f"✅ Added {len(text_descriptions)} text-image pairs")
        
    def compute_similarity_matrix(self) -> np.ndarray:
        """Compute cross-modal similarity matrix"""
        return cosine_similarity(self.text_embeddings, self.image_embeddings)
    
    def retrieve_images(self, text_query: np.ndarray, k: int = 5) -> List[Tuple[int, float]]:
        """Retrieve most similar images for text query"""
        similarities = cosine_similarity(text_query.reshape(1, -1), self.image_embeddings)[0]
        top_k_indices = np.argsort(similarities)[::-1][:k]
        return list(zip(top_k_indices, similarities[top_k_indices]))
    
    def retrieve_texts(self, image_query: np.ndarray, k: int = 5) -> List[Tuple[int, float]]:
        """Retrieve most similar texts for image query"""
        similarities = cosine_similarity(image_query.reshape(1, -1), self.text_embeddings)[0]
        top_k_indices = np.argsort(similarities)[::-1][:k]
        return list(zip(top_k_indices, similarities[top_k_indices]))
    
    def corrupt_modality(self, modality: str, corruption_type: str, noise_level: float) -> np.ndarray:
        """Corrupt a specific modality"""
        if modality == 'text':
            embeddings = self.text_embeddings.copy()
        elif modality == 'image':
            embeddings = self.image_embeddings.copy()
        else:
            raise ValueError("Modality must be 'text' or 'image'")
            
        if corruption_type == 'gaussian':
            noise = np.random.normal(0, noise_level, embeddings.shape)
            embeddings += noise
        elif corruption_type == 'swap':
            # Randomly swap embeddings (misalignment)
            indices = np.random.permutation(len(embeddings))
            embeddings = embeddings[indices]
        elif corruption_type == 'zero_out':
            # Zero out random dimensions
            mask = np.random.random(embeddings.shape) < noise_level
            embeddings[mask] = 0
            
        # Renormalize
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        norms = np.maximum(norms, 1e-8)
        embeddings = embeddings / norms
        
        return embeddings

In [ ]:
# Create synthetic multimodal data
def create_multimodal_data(n_pairs: int = 200, embedding_dim: int = 128):
    """Create synthetic text-image paired data"""
    
    # Create semantic clusters (e.g., animals, vehicles, food, etc.)
    categories = ['animals', 'vehicles', 'food', 'buildings', 'nature']
    n_categories = len(categories)
    
    # Generate base embeddings for each category
    category_centers = np.random.normal(0, 1, (n_categories, embedding_dim))
    
    text_embeddings = []
    image_embeddings = []
    text_descriptions = []
    image_labels = []
    
    for i, category in enumerate(categories):
        # Generate paired embeddings for this category
        n_samples = n_pairs // n_categories
        
        # Text embeddings (descriptions)
        text_center = category_centers[i] + np.random.normal(0, 0.2, embedding_dim)
        text_emb, _ = make_blobs(n_samples=n_samples, centers=[text_center], 
                                cluster_std=0.3, random_state=42+i)
        
        # Image embeddings (visual features)
        image_center = category_centers[i] + np.random.normal(0, 0.2, embedding_dim)
        image_emb, _ = make_blobs(n_samples=n_samples, centers=[image_center], 
                                 cluster_std=0.3, random_state=142+i)
        
        text_embeddings.append(text_emb)
        image_embeddings.append(image_emb)
        
        # Create descriptions and labels
        for j in range(n_samples):
            text_descriptions.append(f"A photo of {category}")
            image_labels.append(f"{category}_image_{j}")
    
    # Stack all embeddings
    text_embeddings = np.vstack(text_embeddings)
    image_embeddings = np.vstack(image_embeddings)
    
    # Add some cross-modal alignment (CLIP-like)
    # Make corresponding text-image pairs more similar
    alignment_strength = 0.3
    for i in range(len(text_embeddings)):
        # Average the embeddings to create alignment
        avg_emb = (text_embeddings[i] + image_embeddings[i]) / 2
        text_embeddings[i] = text_embeddings[i] * (1 - alignment_strength) + avg_emb * alignment_strength
        image_embeddings[i] = image_embeddings[i] * (1 - alignment_strength) + avg_emb * alignment_strength
    
    return text_embeddings, image_embeddings, text_descriptions, image_labels, categories

# Create multimodal data
text_emb, image_emb, text_desc, image_labels, categories = create_multimodal_data()

print(f"📝 Created {len(text_desc)} text-image pairs")
print(f"📊 Categories: {categories}")
print(f"🔢 Embedding dimension: {text_emb.shape[1]}")

In [ ]:
# Initialize CLIP system
clip_system = SimpleCLIPSystem()
clip_system.add_paired_data(text_emb, image_emb, text_desc, image_labels)

# Test clean retrieval
test_text_idx = 0
test_text_emb = text_emb[test_text_idx:test_text_idx+1]
test_image_idx = 0
test_image_emb = image_emb[test_image_idx:test_image_idx+1]

print("\n🔍 Clean Multimodal Retrieval:")
print(f"Query text: {text_desc[test_text_idx]}")

# Text-to-image retrieval
image_results = clip_system.retrieve_images(test_text_emb, k=3)
print("Top 3 retrieved images:")
for rank, (img_idx, score) in enumerate(image_results, 1):
    print(f"  {rank}. {image_labels[img_idx]} (score: {score:.4f})")

# Image-to-text retrieval
text_results = clip_system.retrieve_texts(test_image_emb, k=3)
print("\nTop 3 retrieved texts:")
for rank, (txt_idx, score) in enumerate(text_results, 1):
    print(f"  {rank}. {text_desc[txt_idx]} (score: {score:.4f})")

# Show similarity matrix
similarity_matrix = clip_system.compute_similarity_matrix()
print(f"\n📊 Similarity matrix shape: {similarity_matrix.shape}")
print(f"🔥 Max similarity: {similarity_matrix.max():.4f}")
print(f"🥶 Min similarity: {similarity_matrix.min():.4f}")

## 2. Break It: Modality Misalignment & Corruption

Now let's systematically break the multimodal alignment.

In [ ]:
def evaluate_multimodal_performance(
    system: SimpleCLIPSystem, 
    n_test_queries: int = 50
) -> Dict[str, float]:
    """Evaluate multimodal retrieval performance"""
    
    text_to_image_precisions = []
    image_to_text_precisions = []
    
    # Test text-to-image retrieval
    for i in range(min(n_test_queries, len(system.text_embeddings))):
        query_emb = system.text_embeddings[i:i+1]
        results = system.retrieve_images(query_emb, k=5)
        
        # Check if correct image is in top results (simplified: same index)
        correct_retrieved = any(idx == i for idx, _ in results)
        text_to_image_precisions.append(1.0 if correct_retrieved else 0.0)
    
    # Test image-to-text retrieval
    for i in range(min(n_test_queries, len(system.image_embeddings))):
        query_emb = system.image_embeddings[i:i+1]
        results = system.retrieve_texts(query_emb, k=5)
        
        # Check if correct text is in top results
        correct_retrieved = any(idx == i for idx, _ in results)
        image_to_text_precisions.append(1.0 if correct_retrieved else 0.0)
    
    return {
        'text_to_image_recall@5': np.mean(text_to_image_precisions),
        'image_to_text_recall@5': np.mean(image_to_text_precisions),
        'cross_modal_alignment': (np.mean(text_to_image_precisions) + np.mean(image_to_text_precisions)) / 2
    }

# Test clean performance
clean_performance = evaluate_multimodal_performance(clip_system)
print("\n✨ Clean Multimodal Performance:")
for metric, value in clean_performance.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
# Test different corruption scenarios
corruption_scenarios = [
    {'modality': 'text', 'type': 'gaussian', 'level': 0.1, 'name': 'Text Noise'},
    {'modality': 'image', 'type': 'gaussian', 'level': 0.1, 'name': 'Image Noise'},
    {'modality': 'text', 'type': 'swap', 'level': 1.0, 'name': 'Text-Image Misalignment'},
    {'modality': 'image', 'type': 'zero_out', 'level': 0.3, 'name': 'Image Feature Loss'},
    {'modality': 'text', 'type': 'zero_out', 'level': 0.3, 'name': 'Text Feature Loss'}
]

corruption_results = []

for scenario in corruption_scenarios:
    print(f"\n🧪 Testing {scenario['name']}:")
    
    # Create corrupted system
    corrupted_system = SimpleCLIPSystem()
    
    if scenario['modality'] == 'text':
        corrupted_text = clip_system.corrupt_modality(
            'text', scenario['type'], scenario['level']
        )
        corrupted_system.add_paired_data(
            corrupted_text, clip_system.image_embeddings, 
            text_desc, image_labels
        )
    else:
        corrupted_image = clip_system.corrupt_modality(
            'image', scenario['type'], scenario['level']
        )
        corrupted_system.add_paired_data(
            clip_system.text_embeddings, corrupted_image,
            text_desc, image_labels
        )
    
    # Evaluate performance
    performance = evaluate_multimodal_performance(corrupted_system)
    
    result = {
        'scenario': scenario['name'],
        'modality': scenario['modality'],
        'corruption_type': scenario['type'],
        **performance
    }
    corruption_results.append(result)
    
    print(f"  Cross-modal alignment: {performance['cross_modal_alignment']:.4f}")
    print(f"  Text→Image recall: {performance['text_to_image_recall@5']:.4f}")
    print(f"  Image→Text recall: {performance['image_to_text_recall@5']:.4f}")

# Convert to DataFrame
corruption_df = pd.DataFrame(corruption_results)
print("\n📊 Collected corruption test results")

## 3. Visualize: Embedding Space t-SNE Plots

Let's visualize how corruption affects the multimodal embedding space.

In [ ]:
# Create t-SNE visualizations
def plot_multimodal_embeddings(
    text_emb: np.ndarray, 
    image_emb: np.ndarray, 
    categories: List[str],
    title: str,
    n_samples_per_category: int = 20
) -> None:
    """Plot t-SNE visualization of multimodal embeddings"""
    
    # Sample embeddings for visualization
    sampled_text = []
    sampled_image = []
    labels = []
    modalities = []
    
    samples_per_cat = n_samples_per_category
    for i, category in enumerate(categories):
        start_idx = i * (len(text_emb) // len(categories))
        end_idx = start_idx + samples_per_cat
        
        sampled_text.extend(text_emb[start_idx:end_idx])
        sampled_image.extend(image_emb[start_idx:end_idx])
        
        labels.extend([category] * samples_per_cat)
        labels.extend([category] * samples_per_cat)
        modalities.extend(['text'] * samples_per_cat)
        modalities.extend(['image'] * samples_per_cat)
    
    # Combine embeddings
    combined_emb = np.vstack([sampled_text, sampled_image])
    
    # Apply t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    emb_2d = tsne.fit_transform(combined_emb)
    
    # Create plot
    plt.figure(figsize=(12, 8))
    
    # Plot text embeddings
    text_mask = np.array(modalities) == 'text'
    plt.scatter(emb_2d[text_mask, 0], emb_2d[text_mask, 1], 
               c=[plt.cm.tab10(i) for i in range(len(categories)) for _ in range(samples_per_cat)],
               marker='s', s=100, alpha=0.7, label='Text', edgecolors='black', linewidth=1)
    
    # Plot image embeddings
    image_mask = np.array(modalities) == 'image'
    plt.scatter(emb_2d[image_mask, 0], emb_2d[image_mask, 1], 
               c=[plt.cm.tab10(i) for i in range(len(categories)) for _ in range(samples_per_cat)],
               marker='o', s=100, alpha=0.7, label='Image', edgecolors='black', linewidth=1)
    
    # Add legend
    legend_elements = [plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='gray', 
                                  markersize=10, label='Text'),
                      plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='gray', 
                                  markersize=10, label='Image')]
    
    for i, category in enumerate(categories):
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                         markerfacecolor=plt.cm.tab10(i), markersize=8, 
                                         label=category.title()))
    
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot clean embeddings
plot_multimodal_embeddings(
    clip_system.text_embeddings, 
    clip_system.image_embeddings,
    categories,
    'Clean Multimodal Embeddings: Text vs Image Alignment'
)

In [ ]:
# Plot corrupted embeddings comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Phase 4.2: Multimodal Fusion Breakdown Analysis', fontsize=16, fontweight='bold')

# Plot 1: Performance degradation by corruption type
ax1 = axes[0, 0]
corruption_df = corruption_df.sort_values('cross_modal_alignment')
bars = ax1.barh(corruption_df['scenario'], corruption_df['cross_modal_alignment'])
ax1.axvline(y=clean_performance['cross_modal_alignment'], color='red', linestyle='--', alpha=0.7,
            label=f'Clean Baseline ({clean_performance["cross_modal_alignment"]:.4f})')
ax1.set_xlabel('Cross-Modal Alignment Score')
ax1.set_title('Performance Degradation by Corruption Type')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='x')

# Plot 2: Modality-specific effects
ax2 = axes[0, 1]
modality_effects = corruption_df.groupby('modality')['cross_modal_alignment'].mean()
bars2 = ax2.bar(['Text Corruption', 'Image Corruption'], modality_effects.values)
ax2.axhline(y=clean_performance['cross_modal_alignment'], color='red', linestyle='--', alpha=0.7,
            label=f'Clean Baseline ({clean_performance["cross_modal_alignment"]:.4f})')
ax2.set_ylabel('Average Alignment Score')
ax2.set_title('Modality-Specific Effects')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Similarity matrix heatmap (clean vs corrupted)
ax3 = axes[1, 0]
# Show a subset of the similarity matrix
subset_size = 20
clean_sim_subset = clip_system.compute_similarity_matrix()[:subset_size, :subset_size]

# Create corrupted version
corrupted_text = clip_system.corrupt_modality('text', 'swap', 1.0)
corrupted_system = SimpleCLIPSystem()
corrupted_system.add_paired_data(corrupted_text, clip_system.image_embeddings, text_desc, image_labels)
corrupted_sim_subset = corrupted_system.compute_similarity_matrix()[:subset_size, :subset_size]

# Plot difference
diff_matrix = corrupted_sim_subset - clean_sim_subset
sns.heatmap(diff_matrix, ax=ax3, cmap='RdBu_r', center=0, 
            xticklabels=False, yticklabels=False, cbar_kws={'label': 'Similarity Change'})
ax3.set_title('Similarity Matrix Changes (Corrupted - Clean)')
ax3.set_xlabel('Image Index')
ax3.set_ylabel('Text Index')

# Plot 4: Retrieval success rates
ax4 = axes[1, 1]
x = np.arange(len(corruption_df))
width = 0.35

bars1 = ax4.bar(x - width/2, corruption_df['text_to_image_recall@5'], width, 
                label='Text→Image', alpha=0.8)
bars2 = ax4.bar(x + width/2, corruption_df['image_to_text_recall@5'], width,
                label='Image→Text', alpha=0.8)

ax4.axhline(y=clean_performance['text_to_image_recall@5'], color='blue', linestyle=':', alpha=0.7)
ax4.axhline(y=clean_performance['image_to_text_recall@5'], color='orange', linestyle=':', alpha=0.7)

ax4.set_xlabel('Corruption Scenario')
ax4.set_ylabel('Recall@5')
ax4.set_title('Directional Retrieval Performance')
ax4.set_xticks(x)
ax4.set_xticklabels(corruption_df['scenario'], rotation=45, ha='right')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 4. Optimize: Contrastive Learning Improvements

Let's implement techniques to make multimodal systems more robust.

In [ ]:
class RobustMultimodalSystem(SimpleCLIPSystem):
    """Enhanced multimodal system with robustness optimizations"""
    
    def __init__(self, embedding_dim: int = 128, use_contrastive_regularization: bool = False,
                 use_modality_adaptation: bool = False, adaptation_strength: float = 0.1):
        super().__init__(embedding_dim)
        self.use_contrastive_regularization = use_contrastive_regularization
        self.use_modality_adaptation = use_modality_adaptation
        self.adaptation_strength = adaptation_strength
        
    def add_paired_data(self, text_embeddings: np.ndarray, image_embeddings: np.ndarray,
                       text_descriptions: List[str], image_labels: List[str]):
        """Add paired data with robustness preprocessing"""
        super().add_paired_data(text_embeddings, image_embeddings, text_descriptions, image_labels)
        
        if self.use_modality_adaptation:
            self._apply_modality_adaptation()
            
    def _apply_modality_adaptation(self):
        """Adapt embeddings to be more robust to corruption"""
        # Compute cross-modal centroids
        text_centroid = np.mean(self.text_embeddings, axis=0)
        image_centroid = np.mean(self.image_embeddings, axis=0)
        
        # Pull embeddings toward their modality centroids
        self.text_embeddings = (1 - self.adaptation_strength) * self.text_embeddings + \
                              self.adaptation_strength * text_centroid
        self.image_embeddings = (1 - self.adaptation_strength) * self.image_embeddings + \
                               self.adaptation_strength * image_centroid
        
        # Renormalize
        self.text_embeddings = self.text_embeddings / np.linalg.norm(self.text_embeddings, axis=1, keepdims=True)
        self.image_embeddings = self.image_embeddings / np.linalg.norm(self.image_embeddings, axis=1, keepdims=True)
        
    def retrieve_robust(self, query_embedding: np.ndarray, query_modality: str, k: int = 5) -> List[Tuple[int, float]]:
        """Robust retrieval with modality-specific processing"""
        
        if query_modality == 'text':
            # Text to image retrieval
            similarities = cosine_similarity(query_embedding.reshape(1, -1), self.image_embeddings)[0]
            
            if self.use_contrastive_regularization:
                # Boost similarities to images that are well-aligned with text modality
                text_centroid = np.mean(self.text_embeddings, axis=0)
                alignment_scores = cosine_similarity(self.image_embeddings, text_centroid.reshape(1, -1)).flatten()
                similarities = similarities * (1 + 0.1 * alignment_scores)  # Small boost
                
        else:
            # Image to text retrieval
            similarities = cosine_similarity(query_embedding.reshape(1, -1), self.text_embeddings)[0]
            
            if self.use_contrastive_regularization:
                # Boost similarities to texts that are well-aligned with image modality
                image_centroid = np.mean(self.image_embeddings, axis=0)
                alignment_scores = cosine_similarity(self.text_embeddings, image_centroid.reshape(1, -1)).flatten()
                similarities = similarities * (1 + 0.1 * alignment_scores)
        
        top_k_indices = np.argsort(similarities)[::-1][:k]
        return list(zip(top_k_indices, similarities[top_k_indices]))

In [ ]:
# Test robust multimodal systems
robust_configs = [
    {'use_contrastive_regularization': False, 'use_modality_adaptation': False, 'name': 'Baseline'},
    {'use_contrastive_regularization': True, 'use_modality_adaptation': False, 'name': 'Contrastive Regularization'},
    {'use_contrastive_regularization': False, 'use_modality_adaptation': True, 'name': 'Modality Adaptation'},
    {'use_contrastive_regularization': True, 'use_modality_adaptation': True, 'name': 'Full Robust'}
]

robustness_results = []

# Test each configuration against different corruption types
test_corruptions = ['gaussian', 'swap', 'zero_out']

for config in robust_configs:
    print(f"\n🛡️ Testing {config['name']}:")
    
    for corruption in test_corruptions:
        # Create robust system
        robust_system = RobustMultimodalSystem(
            use_contrastive_regularization=config['use_contrastive_regularization'],
            use_modality_adaptation=config['use_modality_adaptation']
        )
        
        # Apply corruption
        if corruption == 'gaussian':
            corrupted_text = clip_system.corrupt_modality('text', 'gaussian', 0.1)
            robust_system.add_paired_data(corrupted_text, clip_system.image_embeddings, text_desc, image_labels)
        elif corruption == 'swap':
            corrupted_text = clip_system.corrupt_modality('text', 'swap', 1.0)
            robust_system.add_paired_data(corrupted_text, clip_system.image_embeddings, text_desc, image_labels)
        else:  # zero_out
            corrupted_image = clip_system.corrupt_modality('image', 'zero_out', 0.2)
            robust_system.add_paired_data(clip_system.text_embeddings, corrupted_image, text_desc, image_labels)
        
        # Evaluate performance
        performance = evaluate_multimodal_performance(robust_system)
        
        result = {
            'config': config['name'],
            'corruption': corruption,
            **performance
        }
        robustness_results.append(result)
        
        print(f"  {corruption}: alignment={performance['cross_modal_alignment']:.4f}")

# Analyze robustness improvements
robust_df = pd.DataFrame(robustness_results)

# Calculate improvements over baseline
baseline_results = robust_df[robust_df['config'] == 'Baseline'].set_index('corruption')
improvement_data = []

for config_name in ['Contrastive Regularization', 'Modality Adaptation', 'Full Robust']:
    config_results = robust_df[robust_df['config'] == config_name].set_index('corruption')
    for corruption in test_corruptions:
        baseline_align = baseline_results.loc[corruption, 'cross_modal_alignment']
        config_align = config_results.loc[corruption, 'cross_modal_alignment']
        improvement = config_align - baseline_align
        improvement_pct = (improvement / baseline_align) * 100 if baseline_align > 0 else 0
        
        improvement_data.append({
            'config': config_name,
            'corruption': corruption,
            'improvement': improvement,
            'improvement_pct': improvement_pct
        })

improvement_df = pd.DataFrame(improvement_data)
print("\n📈 Robustness Improvements:")
print(improvement_df.to_string(index=False))

In [ ]:
# Final visualization: Robustness comparison
plt.figure(figsize=(14, 8))

# Create subplots for each corruption type
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Multimodal Robustness: Configuration Comparison by Corruption Type', fontsize=14, fontweight='bold')

for i, corruption in enumerate(test_corruptions):
    ax = axes[i]
    subset = robust_df[robust_df['corruption'] == corruption]
    
    configs = subset['config']
    alignments = subset['cross_modal_alignment']
    
    bars = ax.bar(configs, alignments, color=['lightcoral', 'skyblue', 'lightgreen', 'gold'])
    ax.set_title(f'{corruption.title()} Corruption')
    ax.set_ylabel('Cross-Modal Alignment')
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Rotate x labels
    ax.tick_params(axis='x', rotation=45)
    
    # Add value labels
    for bar, align in zip(bars, alignments):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{align:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## Key Takeaways

### What We Learned:
1. **Multimodal alignment is fragile** - Small corruptions in one modality severely impact cross-modal retrieval
2. **Misalignment is catastrophic** - Swapping embeddings (perfect misalignment) destroys all alignment
3. **Feature loss hurts differently** - Text and image corruption have asymmetric effects
4. **Robustness techniques help** - Modality adaptation and contrastive regularization can mitigate some issues

### Production Implications:
- **Monitor modality quality** - Implement separate health checks for text and image encoders
- **Use robust alignment** - Consider techniques like momentum encoders or hard negative mining
- **Implement modality-specific processing** - Handle text and image corruption differently
- **Regular alignment validation** - Continuously check cross-modal retrieval performance

### Next Steps:
- Experiment with real CLIP models and datasets
- Test on actual image-text pairs from the web
- Implement production monitoring for multimodal alignment
- Explore advanced techniques like cross-modal attention

---

**Professor's Challenge:** Design a multimodal system that maintains 70% alignment even with 30% of embeddings randomly swapped. What architecture would you use?